# <center> Hands-On : Genetic </center>
##  In this project, we want to implement **Genetic Algorithm** to solve sudoku.


***
## What is Genetic Algorithm?
#### **Genetic Algorithm(GA)** is a search technique used in computing to find true or approximate solutions to optimization and search problems.<br> The basic idea here is to simulate natural selections, where the population is composed of candidate solutions.<br>The main focus is on evolving a population from which strong and diverse candidates can emerge via mutation and crossover.

***
### We are going to:
- Defining the terms of **Gene** and **Chromosome**
- Constructing the initial population
- Defining and implementing **Fitness Function**
- Implementing **Crossover** and **Mutation**
- Running the algorithm to solve sudoku

***
## Importing the requierd libraries

In [1]:
import numpy as np
import random # for generating random numbers
import copy

***
## Reading the inputs from the file
#### You only need to give the name of your test file which is in the SampleSudoku folder.
The format of a sample sudoku table:(0 means empty)<br>
8 0 6 0 0 0 1 0 7<br>
0 0 0 6 0 2 0 0 0<br>
0 5 3 0 0 4 8 0 6<br>
7 0 4 8 0 0 6 3 0<br>
0 0 0 0 0 0 0 9 0<br>
1 0 0 5 0 0 4 0 0<br>
0 0 1 2 0 0 7 0 9<br>
2 0 0 0 9 6 0 0 0<br>
0 7 0 0 1 0 0 8 0<br>


In [2]:
table = [[]for i in range(9)]

def read_input(file_name):
    lines = []
    with open(file_name) as test:
        while (line := test.readline().rstrip()):
            lines.append(line)
    return lines

filename = input()
path = 'SampleSudoku/' + filename
test = read_input(path)

for i in range(9):
    inp = test[i].split(' ')
    inp = list(map(int, inp))
    table[i] = (inp)
# print(table)

test1.txt


***
Here we changed the 0 cells with an unique negative number.

In [3]:
cnt = -1
for i in range(9):
    for j in range(9):
        if table[i][j] == 0:
            table[i][j] = cnt
            cnt -= 1
# table

***
## What is Chromosome?
#### **Chromosome** is a sample solution to the given problem.Here we consider an sudoku table as a chromosome.
## What is Gene?
#### **Gene** is one element position of a chromosome. Here we consider a gene as an empty cell in our sudoku table.
***
## Fitness Function
#### The fitness function simply defined is a function which takes a candidate solution to the problem as input and produces as output how **good** the solution is with respect to the problem in consideration.
## Fitness Function Implementing
####  In sudoku, the objective is to fill a 9×9 grid with digits so that each column, each row, and each of the nine 3×3 subgrids that compose the grid contain all of the digits from 1 to 9.<br>So our fitness function should be related to the number of unique values in each column, each row, and each block.<br>Here we considered the fitness function as the number of unique values in each column, each row, and each block divided by 9 and then we multiply these numbers.<br> So the fitness function of the solution is $ 9^3 = 729 $.
***
## Implementing Chromosome
#### We have a class for modeling the actions that can be done on a chromosome.
- create_rows_cols_blocks(): This method creates the rows, columns, and blocks of a sudoku table to check the conditions.
- calc_fitness(): This method calculates the fitness function of a chromosome. 


In [4]:
class Chromosome:
    def __init__(self, chromosome_):
        self.data = np.array(chromosome_)
        self.fitness = None
    
    def create_rows_cols_blocks(self, mode='P'):
        rows_cols_blocks = []
        for i in range(9):
            rows_cols_blocks.append(self.data[i])

        for i in range(9):
            col = []
            for j in range(9):
                col.append(self.data[j][i])
            rows_cols_blocks.append(col)

        block = []
        for i in range(9):
            block.append(self.data[i][:3])
        for i in range(9):
            block.append(self.data[i][3:6])
        for i in range(9):
            block.append(self.data[i][6:9])

        square = [[]for i in range(9)]
        for i in range(9):
            for j in range(3):
                for k in range(3):
                    square[i].append(block[i*3:(i+1)*3][j][k])
            rows_cols_blocks.append(square[i])
        
        rows_cols_blocks = np.array(rows_cols_blocks)
        if mode == 'N':
            return rows_cols_blocks
        rows_cols_blocks[rows_cols_blocks < 0] *= -1
        return rows_cols_blocks
        
    def calc_fitness(self):
        rows_cols_blocks = self.create_rows_cols_blocks()
        row_dup_sum, col_dup_sum, block_dup_sum = 0, 0, 0
        
        for i in range(9):
            unique, counts = np.unique(rows_cols_blocks[i], return_counts=True)
            row_dup_sum += len(counts)/9
        
        for i in range(9, 18):
            unique, counts = np.unique(rows_cols_blocks[i], return_counts=True)
            col_dup_sum += len(counts)/9
        
        for i in range(18, 27):
            unique, counts = np.unique(rows_cols_blocks[i], return_counts=True)
            block_dup_sum += len(counts)/9
            
        self.fitness = row_dup_sum * col_dup_sum * block_dup_sum
        

***
## Population
#### **Population** is a subset of solutions in the current generation. It can also be defined as a set of chromosomes.
***
## Implementing the Population
#### We have a class for modeling the population and how to select the population for crossover.
- create_init_population(): Creates the initial population that is composed of chromosomes which are constructed completely with random genes.
- calc_fitnesses(): Calculates the fitness function of each chromosome and check if a chromosome is the answer.
- sort_based_on_fitness(): Sort all of the chromosomes based on their fitness.
- give_rank_to_population(): In this method we rank the population according to fitness and then base selection probabilities on rank.


In [5]:
class Population:
    def __init__(self, population_size_):
        self.population = []
        self.population_size = population_size_
    
    def create_init_population(self, init_chromosome):
        for i in range(self.population_size):
            self.population.append(copy.deepcopy(init_chromosome))
        for i in range(self.population_size):
            for j in range(1, -1*cnt):
                self.population[i].data[self.population[i].data == -1*j] = random.randint(-9, -1)
    
    def calc_fitnesses(self):
        for ch in self.population:
            ch.calc_fitness()
            if ch.fitness == pow(9, 3):
                return ch
        return None
    
    def sort_based_on_fitness(self):
        self.population = sorted(self.population, key=lambda ch: ch.fitness)# sort by fitness
        
    def give_rank_to_population(self):
        rank = np.arange(1,self.population_size+1,1)
        prob = np.true_divide(rank, sum(rank))
        self.population = np.random.choice(a=self.population, size=self.population_size, p=prob)
            

***
## Crossover
#### In genetic algorithms and evolutionary computation, crossover, also called recombination, is a genetic operator used to combine the genetic information of two parents to generate new offspring. Crossover is usually applied in a GA with a high probability $(p_c)$ .<br> For crossovering the parents, we specify a random point in each row، then we put the first part from the first parent and the second part from the second parent to produce a new child.<br><br>[See this link for a better understanding!](https://www.geeksforgeeks.org/crossover-in-genetic-algorithm) <br>  

In [6]:
def crossover(parent1, parent2, P_c=1):
    par1 = copy.deepcopy(parent1.data)
    par2 = copy.deepcopy(parent2.data)
    r = random.uniform(0, 1)
    
    if P_c > r:
        for p1, p2 in zip(par1, par2):
            point = random.randint(0, 8)
            for i in range(len(p1)):
                if p1[i] < 0:
                    if i <= point:
                        p1[i], p2[i] = p2[i], p1[i]

    child1 = Chromosome(par1)
    child2 = Chromosome(par2)
    return child1, child2

***
## Mutation
#### Mutation can be defined as a small random tweak in the chromosome, to get a new solution. It is used to maintain and introduce diversity in the genetic population and is usually applied with a low probability $(p_m)$.<br> For mutating each empty cell(gene), we go through each row, column, and block to identify a list of numbers that can be placed in that cell. Then we choose a number from that list randomly.<br> 

In [7]:
def mutation(chromosome, P_m=1, add_random=False):
    r = random.uniform(0, 1)
    if P_m < r:
        return
    
    rows_cols_blocks = chromosome.create_rows_cols_blocks(mode='N')
    possibles = []
    for i in rows_cols_blocks:
        POSSIBLE_NUMS = {1, 2, 3, 4, 5, 6, 7, 8, 9}
        for j in i:
            if j in POSSIBLE_NUMS:
                POSSIBLE_NUMS.remove(j)
            if -j in POSSIBLE_NUMS:
                POSSIBLE_NUMS.remove(-j)
                
        possibles.append(POSSIBLE_NUMS)
    cell_poss = {}
    for i in range(len(chromosome.data)):
        for j in range(len(chromosome.data[i])):
            if chromosome.data[i][j] < 0:
                poss = {1, 2, 3, 4, 5, 6, 7, 8, 9}
                poss = poss.intersection(possibles[i])
                poss = poss.intersection(possibles[9+j])
                if (i < 3) and (j < 3):
                    poss = poss.intersection(possibles[18])
                if (2 < i < 6) and (j < 3):
                    poss = poss.intersection(possibles[19])
                if (5 < i < 9) and (j < 3):
                    poss = poss.intersection(possibles[20])
                if (i < 3) and (2 < j < 6):
                    poss = poss.intersection(possibles[21])
                if (2 < i < 6) and (2 < j < 6):
                    poss = poss.intersection(possibles[22])
                if (5 < i < 9) and (2 < j < 6):
                    poss = poss.intersection(possibles[23])
                if (i < 3) and (5 < j < 9):
                    poss = poss.intersection(possibles[24])
                if (2 < i < 6) and (5 < j < 9):
                    poss = poss.intersection(possibles[25])
                if (5 < i < 9) and (5 < j < 9):
                    poss = poss.intersection(possibles[26])
                
                random_number = []
                random_number.append(random.randint(1, 9))
                if add_random:
                    poss = poss.intersection(random_number)
                cell_poss[chromosome.data[i][j]] = poss
                li = list(poss)
                if len(li) > 0:
                    index = random.randint(1, len(li))
                    chromosome.data[i][j] = -li[index-1]
                    possibles[i].remove(li[index-1])            

***
## Running The Gentic Algorithm
#### First, we initialized the population size and then we created the first population randomly.

In [8]:
POPULATION_SIZE = 1000
init_ch = Chromosome(table)
init_pop = Population(POPULATION_SIZE)
init_pop.create_init_population(init_ch)
init_pop.calc_fitnesses()
init_pop.sort_based_on_fitness()

#### We run the main algorithm here that is consists of:
We iteratively go through these steps:
- Selecting parents for crossover based on chromosomes ranks
- Crossovering each pair of parents with the probability of $(p_c)$
- Mutating each children with the probabilty of $(p_m)$
- Calculating fitness for each chromosomes for this new generation
- Sorting the new generation based on their fitness
- If the fitness of a chromosome equals to $ 9^3 $, it means that we have reached the answer.

In [9]:
def genetic(init_pop, generation_size):
    generation = []
    generation.append(init_pop)
    for k in range(generation_size):
        i = 0
        childs = []
        generation[k].give_rank_to_population()
        while(i < POPULATION_SIZE):
            ch1, ch2 = crossover(generation[k].population[i], generation[k].population[i+1], 0.9)
            childs.append(ch1)
            childs.append(ch2)
            i += 2

        for j in childs:
            mutation(j, 1 - k/generation_size)
            
        pop = Population(POPULATION_SIZE)
        pop.population = childs
        solution = pop.calc_fitnesses()
        if solution != None:
            print(f"Generation {k} -->  fitness : {solution.fitness}")
            print(f"The answer found in generation {k}")
            return solution

        pop.sort_based_on_fitness()
        generation.append(pop)
        print(f"Generation {k} -->  fitness : {pop.population[-1].fitness}")

In [10]:
%%time
GENERATION_SIZE = 100
solution = genetic(init_pop, GENERATION_SIZE)

Generation 0 -->  fitness : 424.60905349794234
Generation 1 -->  fitness : 511.9012345679012
Generation 2 -->  fitness : 540.4444444444446
Generation 3 -->  fitness : 586.4197530864199
Generation 4 -->  fitness : 610.0850480109739
Generation 5 -->  fitness : 626.1399176954735
Generation 6 -->  fitness : 667.5445816186558
Generation 7 -->  fitness : 684.6666666666667
Generation 8 -->  fitness : 693.5528120713307
Generation 9 -->  fitness : 711.0
Generation 10 -->  fitness : 729.0
The answer found in generation 10
CPU times: user 10.8 s, sys: 108 ms, total: 10.9 s
Wall time: 10.9 s


***
## Printing the solution

In [11]:
solution.data[solution.data < 0] *= -1
for i in solution.data:
    for j in i:
        print(j, end='   ')
    print()

8   2   6   9   3   5   1   4   7   
4   1   7   6   8   2   9   5   3   
9   5   3   1   7   4   8   2   6   
7   9   4   8   2   1   6   3   5   
5   6   8   3   4   7   2   9   1   
1   3   2   5   6   9   4   7   8   
3   4   1   2   5   8   7   6   9   
2   8   5   7   9   6   3   1   4   
6   7   9   4   1   3   5   8   2   


***
#### **1. The method used for selecting chromosomes to crossover:**
We give each chromosome a probability based on their fitness, for example, the chromosome which has the smallest fitness will get 1/(1+2 +...+ PopulationSize) and the chromosome which has the biggest fitness, will get PopulationSize/(1+2 +...+ PopulationSize). Then I choose a chromosome based on these probabilities.
####  - **Why I used this method:**
With this method, the probability of choosing chromosomes with better fitness is more than choosing them randomly. Also we have the diversity too.

***
#### **2. The reason for choosing the fitness function I used:**
Here I considered the fitness function as the number of unique values in each column, each row, and each block divided by 9 and then we multiply these numbers.<br> So the fitness function of the solution is $ 9^3 = 729 $. <br>
Because in the solution, we should have unique values in each row, column, and block, therefore with this fitness function, we can define how much a chromosome is near to the solution.

***
#### **3. The effects of Crossover and Mutation functions:**
**Crossover** is used to create new solutions from the population's genetic information. So with Crossover, we can have a new population with better chromosomes. (To get closer to the answer)<br>
**Mutation** is used to give diversity into the population, means that you avoid premature convergence on a local maximum or minimum. 
#### - **The probability of Crossover and Mutation and the reason for choosing them:**
**Crossover**: We have considered $p_c$ as the constant value of 0.9, which means that we crossover the 90 percent of the population each time.<br>**Reason:** With this crossover rate it can be gauranteed that we have generations that are very diverse.<br>
**Mutation**: We have considered $p_m$ a dynamic value that start with the value of 1 and decreases after producing every a new generation.<br>**Reason:** The mutation I used is a logical way to solve sudoku, so if the more chromosomes we mutate, the better the results are.


***
#### **4. The problem of getting stuck in local minimum or maximum :**
- **Reason:** Having similar chromosomes
- **Problems:** Because we produce similar chromosomes, it's hard to get out of that local  minimum or maximum
- **Solutions:**
    1. Mutation: With mutation we can get close to a different local minimum or maximum.
    2. Bigger popualtion size: Bigger population size will cause more variety.

***
## Conclusion
#### Genetic algorithms are good at taking large search spaces and navigating them, looking for optimal combinations of things.<br>Among the benefits of this algorithm, the following can be mentioned:
- easy to understand
- we have always get an answer
- we can speed up it by changing its hyperparameters
- with the conditions of the problem, we can optimize our algorithm
